# First Steps with the DC Motor Controller

## Preliminaries

The next lines setup some things and import the various libraries required to run this notebook.

In [ ]:
import sys
sys.path.append('..')

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [ ]:
from ctrl.block.linear import ShortCircuit, Affine, Differentiator
from ctrl.block.logger import Logger

Modify the variables `HOST` and `PORT` to reflect the network address of your device:

In [ ]:
from ctrl.client import Controller
#HOST, PORT = "localhost", 9999
HOST, PORT = "192.168.10.51", 9999
device = Controller(host = HOST, port = PORT)

You will communicate with the hardware device through the variable `device`.

The next line will reset the device to make sure you have a clear start and print the initial device configuration.

In [ ]:
device.reset()
print(device.info('all'))

## Making the DC motor move

In order to make the DC motor move you should set the signal *motor1* to 100% for 5 seconds. Use `set_signal` to do that:

In [ ]:
with device:
    device.set_signal('motor1', 100)
    time.sleep(5)

**IMPORTANT**: After executing the above lines the notebook interface will *freeze* for 5 seconds. You will know when you're done when the \* inside `In[*]` changes into a number and the circle on the top of the notebook turns empty again.
 
The device is always running in the background.

It *starts* as soon as the `with` is executed and *stops* immediatelly when iPython finishes
executing the `with` block.
    
We use `time.sleep(5)` to keep the `with` block running for 5 seconds.
 
**TASKS**: 
1. Modify the the code and set the reference to levels: 75%, 50%, 25% and 0%
2. Try a negative reference
3. Try a reference greater than 100%

**QUESTIONS**:
1. How does the motor respond to changes in the reference?
2. How does the motor respond to small references? Can you explain it?
3. Which physical quantity does *reference* control?

## Attaching a Logger and plotting data

If you want to inspect the data collected when you run the motor you will need to attach a *Logger* to device. The *Loger* is a **sink**, that is, it is a block that has only inputs and produces no output. We use the following code to attach a *Logger*:

In [ ]:
device.add_sink('logger', Logger(auto_reset = True), ['clock', 'encoder1'])

This command means that a *Logger* named *logger* is been attached to the device as a sink and its inputs are the signal *clock* and *encoder1*. This *Logger* will record the time (*clock*) and the motor position (*encoder*).

Let us verify that the *logger* is now part of the device:

In [ ]:
print(device.info('all'))

After attaching the *Logger* we need to run the motor again to generate some data:

In [ ]:
with device:
    device.set_signal('motor1', 100)
    time.sleep(5)

We access the data by *reading* the *sink* named *logger*:

In [ ]:
log = device.read_sink('logger')

The data is imported from the controller as a *numpy array* which we label `log`. 

The array `log` is organized in columns as folows:

clock (s) | encoder1 (cycles)
:----: | :---:
0.0  | 0.0
0.1  | 0.2
... | ... 

We will use matplotlib to plot the data:

In [ ]:
t = log[:,0]
position = log[:,1]

fig1 = plt.figure()
plt.plot(t, position)
# the next lines will make the plot look nicer
plt.xlabel('t (s)')
plt.ylabel('position (cycles)')
plt.grid()

### Calculating the velocity

Once we have imported the data from the controller box we can easily calculate other quantities using numpy.

For example we can calculate and plot the velocity using the following code:

In [ ]:
velocity = np.zeros(position.shape)
velocity[1:] = (position[1:] - position[:-1])/(t[1:]-t[:-1])

fig2 = plt.figure()
plt.plot(t, velocity)
plt.xlabel('t (s)')
plt.ylabel('velocity (cycles)')
plt.grid()

**TASKS**: 
1. Write python code that resets the controller logger, sets the reference at 50% for 2 seconds then at 0% for 1/2 second and -50% for 2 more seconds then import the data and plot the result using matplotlib

**QUESTION**:
1. How does the motor respond to different constant *reference*?

## Creating signals and filters

Instead of calculating the velocity from the position obtained from the Logger you will now add signals and blocks to the controller that perform the same calculation, this time using the hardware.

We first add a signal to hold the velocity:

In [ ]:
device.add_signal('velocity1')

This signal will be connected to the output of a *filter*, in this case a *Differentiator*. *Filters* take input signals and produce output signals. In this case the input to the differentiator are the *clock* and *encoder1* signals, and the output signal is the one you just created:

In [ ]:
device.add_filter('velocity1', Differentiator(), ['clock', 'encoder1'],['velocity1'])

Inspect the device and locate the newly created *signal* and *filter*:

In [ ]:
print(device.info('all'))

In order to have access to the velocity signal we modify the logger by adding one more input signal:

In [ ]:
device.add_sink('logger', Logger(auto_reset = True), ['clock', 'encoder1', 'velocity1'])

After attaching the new components run the motor again to generate some data:

In [ ]:
with device:
    device.set_signal('motor1', 100)
    time.sleep(5)

Then *read* the *logger*:

In [ ]:
log = device.read_sink('logger')

The data is imported from the controller as a *numpy array* which we label `log`. 

The array `log` is organized in columns as folows:

clock (s) | encoder1 (cycles) | velocity (Hz)
:----: | :---: | :---:
0.0  | 0.0 | 0.0
0.1  | 0.2 | 2.0
... | ... | ...

Retrive the data:

In [ ]:
t = log[:,0]
position = log[:,1]
velocity = log[:,2]

**TASKS**: 
1. Write python code to plot the position and velocity using matplotlib

## Using the potentiometer

*Filters* can be used to perform all sorts of operations on signals.

You will now connect the potentiometer signal *analog1* to the motor signal *motor1* using a *ShortCircuit* filter:

In [ ]:
device.add_filter('analog1', ShortCircuit(), ['analog1'], ['motor1'])

In [ ]:
print(device.info('all'))

Then run the controller for 15s while moving the potentiometer in the controller box:

In [ ]:
with device:
    time.sleep(15)

Simply remove the filter to go back to the previous behaviour

In [ ]:
device.remove_filter('analog1')

In [ ]:
print(device.info('all'))

Now use an *Affine* filter instead of a *ShortCircuit*:

In [ ]:
device.add_filter('analog1', Affine(2, -100), ['analog1'], ['motor1'])

In [ ]:
print(device.info('all'))

Then run the controller for 15s while changing the potentiometer in the controller box:

In [ ]:
with device:
    time.sleep(15)

Remove the filter to go back to the previous behaviour

In [ ]:
device.remove_filter('analog1')

**QUESTION**:
1. Explain the difference between the *ShortCircuit* and the *Affine* filters?
2. What does the parameters 2 and -100 represent in the *Affine* filter?

### Interacting with iPython widgets

We can use iPython widgets to interact with the controller.

For example we can create a slider that will set the reference for us.

In [ ]:
from IPython.display import display
import ipywidgets as widgets

def set_reference(value):
    device.set_signal('motor1',value)
    
w = widgets.interactive(set_reference, value=(-100,100))
display(w)

Try moving the slider as you `start` the controller.

In [ ]:
device.set_sink('logger', reset=True)
device.start()
device.set_signal('motor1',0)

Don't forget to manually `stop` the controller since we are not using a `with` block:

In [ ]:
device.stop()